In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from ast import literal_eval


A function for examining model metrics

In [2]:
def calculate_metrics(y_test, y_pred):

    # Calculate overall accuracy
    overall_accuracy = accuracy_score(y_test, y_pred)
    print(f"Overall Accuracy: {overall_accuracy}\n")

    # Calculate precision, recall, and F1-score for each class
    report = classification_report(y_test, y_pred)
    print("\nClassification Report:")
    print(report)

Read in the peak featureset

In [3]:
df = pd.read_csv("../../data/Thomas_peaks.csv")

In [4]:
df.head()

,SpecID,Status,Seq,WaveNumber,Absorbance
0,201210-1-00,Normal,[ 727 1026 1820],"[696.81091, 901.13898, 1443.7357]","[1851.9185, 1746.4041, 1537.9485]"
1,201210-1-01,Normal,[ 403 617 1021 1378 1532 1818 2074],"[475.39862, 621.64008, 897.72211, 1141.6857, 1...","[1998.4773, 2034.2784, 1977.515, 1779.342, 173..."
2,201210-1-02,Normal,[ 412 600 1049 1552 1812 2079],"[481.54898, 610.02277, 916.85651, 1260.5923, 1...","[2214.0876, 2846.9824, 3696.4109, 2195.7212, 2..."
3,201210-1-03,Normal,[ 17 385 591 1047 1411 1590 1884 2052],"[211.61731, 463.09796, 603.87244, 915.48975, 1...","[10350.545, 2536.3599, 3342.7229, 3452.3679, 3..."
4,201210-1-04,Normal,[ 368 616 785 1173 1445 1602 1808 2090],"[451.48065, 620.95673, 736.44647, 1001.5945, 1...","[2277.2156, 2264.5063, 2167.6958, 2238.8494, 2..."


Extract the features and labels and create the training and test sets.

In [5]:
X = df['Absorbance']
y = df['Status']

In [6]:
print(X)

0                       [1851.9185, 1746.4041, 1537.9485]
1       [1998.4773, 2034.2784, 1977.515, 1779.342, 173...
2       [2214.0876, 2846.9824, 3696.4109, 2195.7212, 2...
3       [10350.545, 2536.3599, 3342.7229, 3452.3679, 3...
4       [2277.2156, 2264.5063, 2167.6958, 2238.8494, 2...
                              ...                        
3040    [2013.4814, 1650.8135, 1682.9326, 1691.0598, 1...
3041    [1956.9333, 1602.1025, 1624.1553, 1634.3423, 1...
3042    [1906.7185, 1571.1713, 1579.2836, 1575.1331, 1...
3043    [1850.5511, 1533.9598, 1549.0426, 1545.3933, 1...
3044    [1482.5781, 1507.8912, 1505.2144, 1445.8882, 1...
Name: Absorbance, Length: 3045, dtype: object


Pad the feature lists to make them of equal length.

In [7]:
from keras.preprocessing.sequence import pad_sequences

2024-01-12 20:32:46.021484: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-12 20:32:46.057530: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-12 20:32:46.057576: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-12 20:32:46.058879: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-12 20:32:46.064741: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-12 20:32:46.065494: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [8]:
# Convert to lists
X = X.apply(literal_eval)

# Padding sequences with zeros
X_padded = pad_sequences(X, padding='post', dtype='float32')

In [9]:
print(X_padded)

[[1851.9185 1746.404  1537.9485 ...    0.        0.        0.    ]
 [1998.4773 2034.2784 1977.515  ...    0.        0.        0.    ]
 [2214.0876 2846.9824 3696.411  ...    0.        0.        0.    ]
 ...
 [1906.7185 1571.1713 1579.2836 ...    0.        0.        0.    ]
 [1850.5511 1533.9598 1549.0426 ...    0.        0.        0.    ]
 [1482.5781 1507.8912 1505.2144 ...    0.        0.        0.    ]]


Create the training and test sets.

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=1234)

Select and train the model

In [13]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [14]:
y_pred = model.predict(X_test)
calculate_metrics(y_test, y_pred)

Overall Accuracy: 0.7454844006568144


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.75      0.73      0.74       203
 Hypoglycemia       0.73      0.73      0.73       200
       Normal       0.75      0.77      0.76       206

     accuracy                           0.75       609
    macro avg       0.75      0.75      0.75       609
 weighted avg       0.75      0.75      0.75       609

